In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import matplotlib.pyplot as plt

# Đặt đường dẫn và hằng số
LANDMARKS_TRAIN_PATH = 'data/processed_data/landmarks_train.csv'
MODEL_SAVE_PATH = 'data/models/fnn_model.h5'
CLASSES = [chr(i) for i in range(ord('A'), ord('Z')+1)]
NUM_CLASSES = len(CLASSES)
INPUT_SHAPE = (63,)  # 21 landmarks * 3 tọa độ

# Tải dữ liệu huấn luyện
df_train = pd.read_csv(LANDMARKS_TRAIN_PATH, header=None)
y_train = df_train.iloc[:, 0].values  # Cột đầu tiên: nhãn (ví dụ: 'A', 'B', ...)
X_train = df_train.iloc[:, 1:].values  # Các cột còn lại: landmarks

# Mã hóa nhãn thành số
le = LabelEncoder()
le.fit(CLASSES)
y_train = le.transform(y_train)

# Chia dữ liệu thành tập huấn luyện và tập validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Xây dựng mô hình FNN
model = Sequential([
    Dense(128, activation='relu', input_shape=INPUT_SHAPE),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

# Biên dịch mô hình
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Huấn luyện mô hình
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val))

# Vẽ biểu đồ lịch sử huấn luyện
plt.plot(history.history['accuracy'], label='Độ chính xác huấn luyện')
plt.plot(history.history['val_accuracy'], label='Độ chính xác validation')
plt.xlabel('Epoch')
plt.ylabel('Độ chính xác')
plt.legend()
plt.show()

# Lưu mô hình
model.save(MODEL_SAVE_PATH)